In [ ]:
using DynamicPolynomials
using TSSOS
using MAT
using JuMP
include("./Control_utils.jl")

Main.Control

In [2]:
function CartPoleTO(cartpole_param)
    ## Double inverted pendulum # nearly optimal!
    Ns = cartpole_param.control_param.Ns  # 80
    dof = 1

    lx = cartpole_param.lx
    ly = cartpole_param.ly

    dt = cartpole_param.control_param.dt
    m = cartpole_param.mass
    I = cartpole_param.inertial
    g = -9.81

    xc_0 = cartpole_param.xc_0
    yc_0 = 0.0

    t0 = cartpole_param.theta_0

    @polyvar x[1:dof+1, 1:Ns+1]
    @polyvar y[1:dof, 1:Ns+1]
    @polyvar c[1:dof, 1:Ns+1]
    @polyvar s[1:dof, 1:Ns+1]

    @polyvar vx[1:dof+1, 1:Ns+1]
    @polyvar vy[1:dof, 1:Ns+1]
    @polyvar ca[1:dof, 1:Ns+1]
    @polyvar sa[1:dof, 1:Ns+1]

    @polyvar lam_x[1:dof, 1:Ns] # serial manipulator
    @polyvar lam_y[1:dof, 1:Ns]
    @polyvar tau[1:Ns]


    ns = dof * (Ns+1)
    nf = Ns
    nlam = dof * Ns
    var = append!( reshape(x, ns + Ns + 1),  reshape(y, ns),  reshape(c, ns),  reshape(s, ns),
                   reshape(vx, ns + Ns + 1), reshape(vy, ns), reshape(ca, ns), reshape(sa, ns),
                   reshape(lam_x, nlam), reshape(lam_y, nlam), reshape(tau, nf))

    eq = []
    ineq = []

    f = 0

    # cart states
    append!( eq, [ x[2, 1] - xc_0])
    append!( eq, [vx[2, 1] - 0.0])

    # pole states
    append!( eq, [c[1, 1] - cos(t0)])
    append!( eq, [s[1, 1] - sin(t0)])
    append!( eq, [cos(t0) * lx - sin(t0) * ly + x[1, 1] - xc_0 ])
    append!( eq, [sin(t0) * lx + cos(t0) * ly + y[1, 1] - yc_0 ])

    append!(eq, [vx[1, 1] - 0.0])
    append!(eq, [vy[1, 1] - 0.0])
    append!(eq, [ca[1, 1] - 1.0])
    append!(eq, [sa[1, 1] - 0.0])

    for k in 1:Ns
        # propagation of kinematics
        ## pole
        append!( eq, [c[1, k+1] - (c[1, k] * ca[1, k] - s[1, k] * sa[1, k])] )
        append!( eq, [s[1, k+1] - (s[1, k] * ca[1, k] + c[1, k] * sa[1, k])] )
        append!( eq, [x[1, k+1] - (dt * (c[1, k] * vx[1, k] - s[1, k] * vy[1, k]) + x[1, k])] )
        append!( eq, [y[1, k+1] - (dt * (s[1, k] * vx[1, k] + c[1, k] * vy[1, k]) + y[1, k])] )

        ## cart
        append!( eq, [x[2, k+1] - (x[2, k] + dt * vx[2, k])] )

        ## kinematics constraints:
        append!( eq, [c[1,k+1] * lx - s[1,k+1] * ly + x[1, k+1] - x[2, k+1] ])
        append!( eq, [s[1,k+1] * lx + c[1,k+1] * ly + y[1, k+1] - 0 ])

        append!( eq, [s[1,k+1]^2  + c[1,k+1]^2 - 1.0 ])
        append!( eq, [sa[1,k+1]^2 + ca[1,k+1]^2 - 1.0 ])

        # propagation of sdynamics
        Jx = (-c[1, k+1] * ly - s[1, k+1] * lx)
        Jy = (-s[1, k+1] * ly + c[1, k+1] * lx)
        ## pole
        append!( eq, [(sa[1, k+1] - sa[1, k]) * I - (Jx * lam_x[1, k] + Jy * lam_y[1, k]) * dt^2] )
        append!( eq, [m * vx[1, k+1] - m * ( ca[1, k] * vx[1, k] + sa[1, k] * vy[1, k] ) - (  c[1, k+1]*lam_x[1, k] + s[1, k+1]*lam_y[1, k] + m * s[1, k+1] * g)* dt ] )
        append!( eq, [m * vy[1, k+1] - m * (-sa[1, k] * vx[1, k] + ca[1, k] * vy[1, k] ) - (- s[1, k+1]*lam_x[1, k] + c[1, k+1]*lam_y[1, k] + m * c[1, k+1] * g)* dt ] )

        ## cart
        append!( eq, [m * vx[2, k+1] - m * vx[2, k] - (lam_x[1, k] + tau[k])* dt ] )

        ## cost function
        f = f + ( x[2, k]^2 +  x[1, k]^2 + (y[1, k] - abs(ly))^2 + (c[1, k] - 1)^2 + s[1, k]^2 ) * cartpole_param.run_cost[1]
        f = f + (vx[2, k]^2 + vx[1, k]^2 + vy[1, k]^2 + (ca[1, k] - 1)^2 + sa[1, k]^2) * cartpole_param.run_cost[1]
        f = f + cartpole_param.run_cost[1] * tau[k]^2

    end

    p_goal = [0; dof]
    
    if cartpole_param.terminal_cons
        append!( eq, [ x[2, end]] )
        append!( eq, [vx[2, end]] )
        append!( eq, [ x[1, end]] )
        append!( eq, [vx[1, end]] )
        append!( eq, [ y[1, end] - abs(ly)] )
        append!( eq, [vy[1, end]] )
        append!( eq, [ c[1, end] - 1.0] )
        append!( eq, [ s[1, end]] )
        append!( eq, [ ca[1, end] - 1.0] )
        append!( eq, [ sa[1, end]] )
    else
        f = f + ( x[2, end]^2 +  x[1, end]^2 + (y[1, end] - abs(ly))^2 + (c[1, end] - 1)^2 + s[1, end]^2) * cartpole_param.terminal_cost[1]
        f = f + (vx[2, end]^2 + vx[1, end]^2 + vy[1, end]^2 + (ca[1, end] - 1)^2 + sa[1, end]^2) * cartpole_param.terminal_cost[1]
    end
    
# #     for k = 1:length(var)-Ns
# #     #    append!( ineq, [2 - var[k] ])
# #     #    append!( ineq, [2 + var[k] ])
# # #         append!( ineq, [100 - var[k].^2 ])
# #     end

#     for k = 1:Ns
#        append!( ineq, [20 - tau[k] ])
#        append!( ineq, [20 + tau[k] ])

#        append!( ineq, [50 - lam_x[k] ])
#        append!( ineq, [50 + lam_x[k] ])

#        append!( ineq, [50 - lam_y[k] ])
#        append!( ineq, [50 + lam_y[k] ])
#     end

    for k = 1:Ns+1
       append!( ineq, [0 + ca[1, k] ])
    #    append!( ineq, [1 + ca[1, k] ])
    end

    pop = append!([f], ineq, eq)

    order = 2

    eq_len = length(eq)

    time_start = time()
#     opt,sol,data =cs_tssos_first_export_tsl(pop, var, order, numeq=eq_len, CS = "MF", TS = false, solution=true, QUIET=false, MomentOne=false)
    opt,sol,data =cs_tssos_first(pop, var, order, numeq=eq_len, CS = "MF", TS = false, solution=true, QUIET=false, MomentOne=false)
    time_end = time()
    
    elapsed = time_end - time_start
    
#     flag = primal_status(model)
#     flag = string(flag)
    
#     flag2 = termination_status(model)
#     flag2 = string(flag2)

    
    moment = []
    for k = 1:length(data.moment)
        append!(moment, [convert(Matrix{Float64}, data.moment[k])]) # data.Mmatrix[k])
    end

    return Dict("sol" => sol, #"sol_approx" => sol_approx,
                "Ns" => Ns, "dof" => dof, "Ns" => Ns, "moment" => moment,
                "opt" => opt, "param" => cartpole_param, "elapsed" => elapsed) #,
#                 "primal" => flag, "termination" => flag2)
    
end

CartPoleTO (generic function with 1 method)

In [ ]:
mpc_param      = Control.param_base(0.125, 40, 2)
weights        = ones(11, 1)
cartpole_param = Control.param_cartpole(mpc_param, .1, .1, 0.0, -.1, weights, 200 * weights, false, 0, pi / 2)
cartpole_param.terminal_cost[1]

logger = []
idd = 0
for steps = [80]
    for t0 = [pi / 4, pi * 3 / 4, pi * 0.95] # pi / 2 and pi / 4 is solved.
        for terminal = [false] # [false, true]
            for xc0 = [-0.3, -0.1, 0, 0.1, 0.3]
                for leng = [0.5]
                    if steps == 40
                        dt = 0.25
                    else
                        dt = 0.125 
                    end
                    cartpole_param.control_param.dt = dt
                    cartpole_param.control_param.Ns = steps
                    cartpole_param.theta_0 = t0
                    cartpole_param.terminal_cons = terminal

                    cartpole_param.lx = 0
                    cartpole_param.ly = -leng
                    cartpole_param.mass = leng
                    cartpole_param.inertial = leng

                    cartpole_param.terminal_cons = terminal
                    cartpole_param.xc_0 = xc0
                    idd = idd + 1
                    if idd > 1
                        log = CartPoleTO(cartpole_param)
                        logger = append!(logger, log)
                        
                        matwrite("dec_log_th3_" * string(idd) * ".mat", log)
                    end
                end
            end
        end
    end
end

*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is launching...
-----------------------------------------------------------------------------
The clique sizes of varibles:
[5, 3, 1]
[160, 160, 408]
-----------------------------------------------------------------------------
Obtained the variable cliques in 0.1872851 seconds. The maximal size of cliques is 5.
Assembling the SDP...
There are 3542 affine constraints.
SDP assembling time: 0.9274403 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 3542            
  Cones                  : 0               
  Scalar variables       : 1532            
  Matrix variables       : 728             
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency che

11  8.9e-01  2.2e-03  3.0e-04  8.18e-01   1.132337170e+01   1.131287682e+01   2.2e-03  0.20  
12  5.2e-01  1.3e-03  1.4e-04  9.01e-01   9.527389750e+00   9.520845654e+00   1.3e-03  0.22  
13  1.2e-01  3.1e-04  1.7e-05  9.34e-01   7.280856677e+00   7.279028604e+00   3.1e-04  0.22  
14  2.1e-02  5.1e-05  1.1e-06  9.80e-01   6.541262570e+00   6.540942509e+00   5.1e-05  0.23  
15  3.5e-03  8.8e-06  8.1e-08  9.96e-01   6.414111527e+00   6.414055559e+00   8.8e-06  0.23  
16  7.6e-04  1.9e-06  8.2e-09  9.98e-01   6.393978064e+00   6.393965700e+00   1.9e-06  0.25  
17  1.6e-04  4.1e-07  8.4e-10  9.97e-01   6.389473994e+00   6.389471236e+00   4.1e-07  0.25  
18  4.3e-05  1.1e-07  1.1e-10  9.93e-01   6.388541577e+00   6.388540842e+00   1.1e-07  0.27  
19  1.8e-05  4.4e-08  3.0e-11  9.84e-01   6.388343077e+00   6.388342764e+00   4.4e-08  0.27  
20  7.9e-06  2.2e-08  9.3e-12  1.02e+00   6.388275338e+00   6.388275184e+00   2.0e-08  0.30  
21  4.3e-06  1.1e-08  3.8e-12  9.87e-01   6.388236670e+00   

4   1.5e+02  3.7e-01  5.2e-01  -6.95e-01  -6.277682165e+02  -6.288323287e+02  3.7e-01  0.06  
5   9.4e+01  2.4e-01  1.7e-01  2.58e-03   -3.209256455e+02  -3.210151748e+02  2.4e-01  0.08  
6   5.5e+01  1.4e-01  7.5e-02  7.17e-01   -1.626828076e+02  -1.627099300e+02  1.4e-01  0.11  
7   3.4e+01  8.6e-02  4.5e-02  7.80e-01   3.922628115e+01   3.914024830e+01   8.6e-02  0.13  
8   1.1e+01  2.8e-02  9.2e-03  9.01e-01   6.773654602e+01   6.769657588e+01   2.8e-02  0.13  
9   5.1e+00  1.3e-02  3.4e-03  7.24e-01   3.203135846e+01   3.199602808e+01   1.3e-02  0.16  
10  2.0e+00  5.1e-03  9.6e-04  7.68e-01   1.849199278e+01   1.847176496e+01   5.1e-03  0.19  
11  8.9e-01  2.2e-03  2.9e-04  8.17e-01   1.168589683e+01   1.167541443e+01   2.2e-03  0.19  
12  5.2e-01  1.3e-03  1.3e-04  8.99e-01   9.903262114e+00   9.896729176e+00   1.3e-03  0.20  
13  1.4e-01  3.6e-04  2.1e-05  9.32e-01   7.806637149e+00   7.804555568e+00   3.6e-04  0.23  
14  1.9e-02  4.8e-05  1.1e-06  9.76e-01   6.977196064e+00   

In [4]:
string(2)
# "log.mat" * "2"

"2"